## Finetune on DenseNet-201

In [ ]:
from importlib import reload
import sys
sys.path.insert(0,'..')
import utils; reload(utils)
from utils import *
from glob import glob
from keras.models import Model

In [ ]:
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD, adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard

In [ ]:
train_dir = '/datasets/Google_Landmark_Recognition/train_images'
# TODO - validation
# val_dir = '/datasets/Google_Landmark_Recognition/val_images'

# TODO - Donwload full dataset
num_label = 1193
batch_size = 64
num_of_epochs = 3

## Transfer leraning from DenseNet

In [ ]:
im_width, im_height = 224, 224 #fixed size for DenseNet
# bliock = [6, 12, 48, 32]
base_model = DenseNet201(include_top=False)

In [ ]:
# Add new layer after DenseNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_label, activation='softmax', name='fc_predict')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze all weight in inceptionV3 for transfer learning
for layer in base_model.layers:
    layer.trainable = False

opt = adam(lr=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Data Augmentation

In [ ]:
# TODO - validation

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    rescale=1/255,
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

## Training

In [ ]:
# Plot learning curve on tensor board
tensor_board = TensorBoard(log_dir='/log/tensorboard', batch_size=batch_size)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator, 
                    epochs = num_of_epochs,
                    # validation_data=val_generator,
                    callbacks = [tensor_board])